In [ ]:
!pip install -q pyomo
!pip install -q folium
!apt update
!apt install -y -q coinor-cbc
!apt install -y -q glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,132 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,152 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restri

In [ ]:
import numpy as np
import pandas as pd
import folium
import math
import base64
from pyomo.environ import (ConcreteModel,
                           Var,
                           ConstraintList,
                           Objective,
                           Binary,
                           Reals,
                           PositiveIntegers,
                           SolverFactory)

# initialize solver
solver = SolverFactory("cbc", executable='/usr/bin/cbc')
solver_glpk = solver = SolverFactory('glpk')

In [ ]:
!pip install geopy
!pip install pandas


In [ ]:
import pandas as pd

# For tab-separated values
city_names = pd.read_csv('cities.txt', delimiter='\t', header=None, names=['City'])
city_names = city_names.head(15)

In [ ]:
city_names

,City
0,IIT Bombay Mumbai
1,Versova Beach Mumbai
2,Juhu Beach Mumbai
3,Andheri Sports Complex Mumbai
4,Lokhandwala Market Mumbai
5,Inorbit Mall Mumbai
6,Mindspace Garden Mumbai
7,Andheri Railway Station Mumbai
8,Shivaji Park Mumbai
9,Powai Lake Mumbai


In [ ]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_locator")

def get_lat_lon(city_name):
    try:
        location = geolocator.geocode(city_name)
        return (location.latitude, location.longitude)
    except:
        return (None, None)

city_names['lat'], city_names['lon'] = zip(*city_names['City'].apply(get_lat_lon))

In [ ]:
city_names.to_csv('cities_data.csv', index=False)

In [ ]:
cities_data = pd.read_csv("cities_data.csv")
cities_data

,City,lat,lon
0,IIT Bombay Mumbai,19.132619,72.914970
1,Versova Beach Mumbai,19.130012,72.813325
2,Juhu Beach Mumbai,19.122146,72.818762
3,Andheri Sports Complex Mumbai,19.128273,72.836923
4,Lokhandwala Market Mumbai,19.143998,72.825731
5,Inorbit Mall Mumbai,19.172965,72.835077
6,Mindspace Garden Mumbai,19.161502,73.002494
7,Andheri Railway Station Mumbai,19.119698,72.846420
8,Shivaji Park Mumbai,19.027279,72.838398
9,Powai Lake Mumbai,19.127720,72.906214


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Define the haversine function using math
def haversine(lon1, lat1, lon2, lat2):
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    lon_diff = lon2 - lon1
    lat_diff = lat2 - lat1
    a = math.sin(lat_diff / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(lon_diff / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6357
    return c * r

In [ ]:
# Create a distance matrix
num_cities = len(cities_data)
distance_matrix = np.zeros((num_cities, num_cities))

for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            distance = haversine(cities_data.iloc[i]['lon'], cities_data.iloc[i]['lat'], cities_data.iloc[j]['lon'], cities_data.iloc[j]['lat'])
            distance_matrix[i, j] = round(distance, 1)
        else:
            distance_matrix[i, j] = 0  # Distance to itself is 0

# Convert distance matrix to DataFrame
cities_distances = pd.DataFrame(distance_matrix, index=cities_data['City'], columns=cities_data['City'])

# Save the DataFrame to a CSV file
cities_distances.to_csv('distance_matrix.csv')


In [ ]:
# Save the DataFrame to a CSV file
cities_distances.to_csv('cities_distances.csv')

In [ ]:
cities_distances

City,IIT Bombay Mumbai,Versova Beach Mumbai,Juhu Beach Mumbai,Andheri Sports Complex Mumbai,Lokhandwala Market Mumbai,Inorbit Mall Mumbai,Mindspace Garden Mumbai,Andheri Railway Station Mumbai,Shivaji Park Mumbai,Powai Lake Mumbai,Mahalaxmi Racecourse Mumbai,Saki Naka Mumbai,Oshiwara Mumbai,Dahisar Check Naka Mumbai,Oshiwara Police Station Mumbai
City,,,,,,,,,,,,,,,
IIT Bombay Mumbai,0.0,10.7,10.2,8.2,9.4,9.5,9.7,7.3,14.2,1.1,19.1,4.3,8.7,14.6,8.9
Versova Beach Mumbai,10.7,0.0,1.0,2.5,2.0,5.3,20.1,3.7,11.7,9.7,16.1,7.8,3.1,15.4,2.7
Juhu Beach Mumbai,10.2,1.0,0.0,2.0,2.5,5.9,19.7,2.9,10.7,9.2,15.2,7.0,3.5,16.0,3.1
Andheri Sports Complex Mumbai,8.2,2.5,2.0,0.0,2.1,5.0,17.7,1.4,11.2,7.3,16.0,5.4,2.5,14.8,2.2
Lokhandwala Market Mumbai,9.4,2.0,2.5,2.1,0.0,3.4,18.6,3.5,13.0,8.6,17.6,7.2,1.1,13.5,0.7
Inorbit Mall Mumbai,9.5,5.3,5.9,5.0,3.4,0.0,17.6,6.0,16.2,9.0,20.9,8.8,2.5,10.2,2.8
Mindspace Garden Mumbai,9.7,20.1,19.7,17.7,18.6,17.6,0.0,17.0,22.8,10.8,27.3,13.8,17.7,17.4,18.0
Andheri Railway Station Mumbai,7.3,3.7,2.9,1.4,3.5,6.0,17.0,0.0,10.3,6.3,15.2,4.1,3.6,15.6,3.5
Shivaji Park Mumbai,14.2,11.7,10.7,11.2,13.0,16.2,22.8,10.3,0.0,13.2,5.0,10.2,13.7,25.8,13.4


In [ ]:
cities_demands = pd.read_csv("cities_demands.csv", index_col=0)
cities_demands

,id,name,demand
0,1640184983,IIT Bombay Mumbai,0
1,42942995,Versova Beach Mumbai,-200
2,620427148,Juhu Beach Mumbai,400
3,2630075008,Andheri Sports Complex Mumbai,-200
4,4260270146,Lokhandwala Market Mumbai,-100
5,794424283,Inorbit Mall Mumbai,-50
6,245659921,Mindspace Garden Mumbai,-50
7,2630075004,Andheri Railway Station Mumbai,-300
8,700688007,Shivaji Park Mumbai,250
9,152305308,Powai Lake Mumbai,100


In [ ]:
# # Get the first 5 cities
# cities_demands = cities_demands

In [ ]:
def get_map(cities_data, cities_distances, cities_demand=None, polylines=None, distances=None):
    cities_coords = cities_data.iloc[:,1:3].to_numpy()
    print(cities_coords)
    cities_names = cities_data.iloc[:,0].to_numpy()
    print(cities_names)
    cities_dists = cities_distances.to_numpy()
    print(cities_dists)
    if cities_demand is not None:
        cities_demand = cities_demand.iloc[:, 2].to_numpy()

    min_pt = (cities_data[['lat', 'lon']].min().values).tolist()
    max_pt = (cities_data[['lat', 'lon']].max().values).tolist()

    mymap = folium.Map(location=[18.9690, 72.8777])
    if cities_demand is not None:
        for name, pt, demand in zip(cities_names, cities_coords, cities_demand):
            if demand == 0:
                folium.Marker(list(pt), popup = f'{name} \n {demand}', icon=folium.Icon(color="black", icon="square-parking", prefix="fa")).add_to(mymap)
            elif demand < 0:
                folium.Marker(list(pt), popup = f'{name} \n {demand}', icon=folium.Icon(color="red", icon="truck-pickup", prefix="fa")).add_to(mymap)
            else:
                folium.Marker(list(pt), popup = f'{name} \n {demand}', icon=folium.Icon(color="green", icon="truck-pickup", prefix="fa")).add_to(mymap)
    else:
        for name, pt in zip(cities_names, cities_coords):
            folium.Marker(list(pt), popup = name).add_to(mymap)

    mymap.fit_bounds([min_pt, max_pt])
    if polylines is not None:
        for i in range(len(polylines)):
            line = polylines[i]
            # Calculate the bearing between the two points
            start_point = line[0]
            end_point = line[-1]
            lat1, lon1 = start_point
            lat2, lon2 = end_point
            y = math.sin(lon2-lon1) * math.cos(lat2)
            x = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(lon2-lon1)
            bearing = math.atan2(y, x) * 180 / math.pi
            # Calculate the midpoint of the polyline
            midpoint = [(line[0][0] + line[1][0]) / 2, (line[0][1] + line[1][1]) / 2]
            if distances is not None:
                folium.PolyLine(line, tooltip=distances[i], popup=distances[i], weight=6, color="#111").add_to(mymap)
            else:
                folium.PolyLine(line, weight=1, color="#111").add_to(mymap)

            # Add an arrow marker to the end of the polyline with the correct rotation
            folium.RegularPolygonMarker(
                location=midpoint,
                color="#111",
                fill_color="#111",
                number_of_sides=3,
                opacity=1,
                radius=10,
                rotation=bearing-90,
                tooltip=''
            ).add_to(mymap)
    return mymap

In [ ]:
mymap = get_map(cities_data, cities_distances, cities_demands)
#mymap = get_map(cities_data, cities_distances)
mymap

[[19.13261855 72.9149702 ]
 [19.1300123  72.81332508]
 [19.12214645 72.81876151]
 [19.12827285 72.83692281]
 [19.1439977  72.82573052]
 [19.1729654  72.83507694]
 [19.16150185 73.00249366]
 [19.1196976  72.8464205 ]
 [19.02727875 72.83839775]
 [19.1277203  72.90621424]
 [18.9852045  72.82059836]
 [19.1082208  72.8835824 ]
 [19.1502437  72.8342294 ]
 [19.2578063  72.8718591 ]
 [19.1475973  72.831659  ]]
['IIT Bombay Mumbai' 'Versova Beach Mumbai ' 'Juhu Beach Mumbai'
 'Andheri Sports Complex Mumbai' 'Lokhandwala Market Mumbai'
 'Inorbit Mall Mumbai' 'Mindspace Garden Mumbai'
 'Andheri Railway Station Mumbai' 'Shivaji Park Mumbai'
 'Powai Lake Mumbai' 'Mahalaxmi Racecourse Mumbai' 'Saki Naka Mumbai'
 'Oshiwara Mumbai' 'Dahisar Check Naka Mumbai'
 'Oshiwara Police Station Mumbai']
[[ 0.  10.7 10.2  8.2  9.4  9.5  9.7  7.3 14.2  1.1 19.1  4.3  8.7 14.6
   8.9]
 [10.7  0.   1.   2.5  2.   5.3 20.1  3.7 11.7  9.7 16.1  7.8  3.1 15.4
   2.7]
 [10.2  1.   0.   2.   2.5  5.9 19.7  2.9 10.7  9.2

In [ ]:
cities_coords = cities_data.iloc[:, 1:3].values
#print(cities_coords)
cities_names = cities_data.iloc[:, 0].values
#print(cities_names)
demand_list = cities_demands.iloc[:, 2].values
print(demand_list)
distance_matrix = cities_distances.to_numpy()
truck_capacity = 600
print("shape of distance matrix: ", distance_matrix.shape)

[   0 -200  400 -200 -100  -50  -50 -300  250  100   50  -50  200  100
  100]
shape of distance matrix:  (15, 15)


In [ ]:
def PDTSP1Model(distance_matrix, demand_list, vehicle_capacity, depot_index):
#def PDTSP1Model(distance_matrix, depot_index):
    n = len(distance_matrix)
    demand_list[depot_index] = -sum(demand_list)

    model = ConcreteModel(name="1-PDTSP")

    model.x = Var(range(n), range(n), initialize=0.0, domain=Binary)
    model.f = Var(range(n), range(n), initialize=np.inf, domain=Reals)
    model.u = Var(range(n), domain=PositiveIntegers)

    model.objective = Objective(expr=sum(distance_matrix[i, j]*model.x[i, j] for i in range(n) for j in range(n)))

    model.constraints = ConstraintList()
    for i in range(n):
        model.constraints.add(sum(model.x[i, k] for k in range(n) if i != k) == 1)

    for j in range(n):
        model.constraints.add(sum(model.x[k, j] for k in range(n) if j != k) == 1)

    model.constraints.add(model.u[0] == 1)
    for i in range(1, n):
        model.u[i].setlb(2)
        model.u[i].setub(n)

    for i in range(n):
        for j in range(n):
            if i != 0 and j != 0:
                model.constraints.add(model.u[i] - model.u[j] + 1 <= (n-1)*(1-model.x[i, j]))

    for i in range(n):
        model.constraints.add(sum(
            model.f[i, k] for k in range(n) if i != k) - sum(
                model.f[k, i] for k in range(n) if i != k) == demand_list[i])

    for i in range(n):
        for j in range(n):
            model.constraints.add(model.f[i, j] >= 0)
            model.constraints.add(model.f[i, j] <= vehicle_capacity*model.x[i, j])
    return model

In [ ]:
model = PDTSP1Model(distance_matrix, demand_list, truck_capacity, depot_index=0)
#model = PDTSP1Model(distance_matrix, depot_index=0)
model.pprint()

3 Var Declarations
    f : Size=225, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}*{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (0, 0) :  None :   inf :  None : False : False :  Reals
          (0, 1) :  None :   inf :  None : False : False :  Reals
          (0, 2) :  None :   inf :  None : False : False :  Reals
          (0, 3) :  None :   inf :  None : False : False :  Reals
          (0, 4) :  None :   inf :  None : False : False :  Reals
          (0, 5) :  None :   inf :  None : False : False :  Reals
          (0, 6) :  None :   inf :  None : False : False :  Reals
          (0, 7) :  None :   inf :  None : False : False :  Reals
          (0, 8) :  None :   inf :  None : False : False :  Reals
          (0, 9) :  None :   inf :  None : False : False :  Reals
         (0, 10) :  None :   inf :  None : False : False :  Reals
         (0, 11) :  None :   inf :  None : False : False :  Rea

In [ ]:
print("Problem Details")
print("."*100)
print("Number of Cities: ", len(distance_matrix))
print("Number of Delivery Points: ", len(demand_list[np.where(demand_list < 0)]))
print("Number of Pickup Points: ", len(demand_list[np.where(demand_list > 0)]))
print("="*100)
print("Model Details: ")
print("."*100)
print("Total Number of variables: ", len(model.x) + len(model.u) + len(model.f))
print("Total Number of constraints: ", len(model.constraints) + 2*len(model.u) + len(model.x) )

Problem Details
....................................................................................................
Number of Cities:  15
Number of Delivery Points:  8
Number of Pickup Points:  7
Model Details: 
....................................................................................................
Total Number of variables:  465
Total Number of constraints:  947


In [ ]:
%%time
## CBC Solver
result = solver.solve(model)
print("Solver Status: ", result.solver.status)
print("Solver Termination Condition: ", result.solver.termination_condition)

Solver Status:  ok
Solver Termination Condition:  optimal
CPU times: user 586 ms, sys: 27.4 ms, total: 614 ms
Wall time: 1.87 s


In [ ]:
print("optimal objective value: ", model.objective())

optimal objective value:  82.0


In [ ]:
sol_edges = []
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        if model.x[i, j].value:
            sol_edges.append((j, i))

In [ ]:
polylines = []
for edge in sol_edges:
    inp, out = edge
    inp_coordinate = cities_coords[inp]
    out_coordinate = cities_coords[out]
    polylines.append((inp_coordinate, out_coordinate))

In [ ]:
sol_dict = {key:val for key, val in sol_edges}
optimal_path_idx = []
c = 0
idx = 0
while c < len(distance_matrix):
    optimal_path_idx.append(idx)
    idx = sol_dict[idx]
    c += 1
optimal_path_idx.append(0)

In [ ]:
optimal_path = [cities_names[i] for i in optimal_path_idx]
for city in optimal_path:
    print(city, " -> ", end="")

IIT Bombay Mumbai  -> Mindspace Garden Mumbai  -> Dahisar Check Naka Mumbai  -> Inorbit Mall Mumbai  -> Oshiwara Mumbai  -> Oshiwara Police Station Mumbai  -> Lokhandwala Market Mumbai  -> Versova Beach Mumbai   -> Juhu Beach Mumbai  -> Andheri Sports Complex Mumbai  -> Andheri Railway Station Mumbai  -> Mahalaxmi Racecourse Mumbai  -> Shivaji Park Mumbai  -> Saki Naka Mumbai  -> Powai Lake Mumbai  -> IIT Bombay Mumbai  -> 

In [ ]:
distances = []
for i in range(len(optimal_path_idx)-1):
    distances.append(f"{distance_matrix[optimal_path_idx[i], optimal_path_idx[i+1]] :.2f} K.M.")

In [ ]:
mymap = get_map(cities_data, cities_distances, cities_demands, polylines, distances)
#mymap = get_map(cities_data, cities_distances, polylines, distances)
mymap.save("map2.html")
mymap

[[19.13261855 72.9149702 ]
 [19.1300123  72.81332508]
 [19.12214645 72.81876151]
 [19.12827285 72.83692281]
 [19.1439977  72.82573052]
 [19.1729654  72.83507694]
 [19.16150185 73.00249366]
 [19.1196976  72.8464205 ]
 [19.02727875 72.83839775]
 [19.1277203  72.90621424]
 [18.9852045  72.82059836]
 [19.1082208  72.8835824 ]
 [19.1502437  72.8342294 ]
 [19.2578063  72.8718591 ]
 [19.1475973  72.831659  ]]
['IIT Bombay Mumbai' 'Versova Beach Mumbai ' 'Juhu Beach Mumbai'
 'Andheri Sports Complex Mumbai' 'Lokhandwala Market Mumbai'
 'Inorbit Mall Mumbai' 'Mindspace Garden Mumbai'
 'Andheri Railway Station Mumbai' 'Shivaji Park Mumbai'
 'Powai Lake Mumbai' 'Mahalaxmi Racecourse Mumbai' 'Saki Naka Mumbai'
 'Oshiwara Mumbai' 'Dahisar Check Naka Mumbai'
 'Oshiwara Police Station Mumbai']
[[ 0.  10.7 10.2  8.2  9.4  9.5  9.7  7.3 14.2  1.1 19.1  4.3  8.7 14.6
   8.9]
 [10.7  0.   1.   2.5  2.   5.3 20.1  3.7 11.7  9.7 16.1  7.8  3.1 15.4
   2.7]
 [10.2  1.   0.   2.   2.5  5.9 19.7  2.9 10.7  9.2